In [39]:
from pathlib import Path 

import numpy as np
import pandas as pd
import sqlalchemy as db
from tqdm.auto import tqdm
from utils import parse_filename

In [2]:
username = 'amos'
password = 'M0$hicat'
host = '192.168.0.131'
port = '3306'
database = 'CineFace'

In [3]:
connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'
engine = db.create_engine(connection_string)
conn = engine.connect()

In [4]:
encodings_dir = Path('./data/encodings')
subdirs = [x for x in encodings_dir.iterdir()]
data = []
for subdir in tqdm(subdirs, leave=True):
    files = [x for x in subdir.iterdir() if x.suffix == '.npy']
    for file in tqdm(files, leave=False):
        e = np.load(str(file))
        datum = {'filepath': str(file),
                 'encoding': e}
        data.append(datum)
df = pd.DataFrame(data)
df.head()

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/249535 [00:00<?, ?it/s]

  0%|          | 0/462487 [00:00<?, ?it/s]

,filepath,encoding
0,data/encodings/homeland_2011_796960/S05E04_390...,"[-0.11495646834373474, 0.09172723442316055, 0...."
1,data/encodings/homeland_2011_796960/S06E05_389...,"[-0.026263369247317314, 0.03598316013813019, 0..."
2,data/encodings/homeland_2011_796960/S01E01_429...,"[-0.2073737531900406, 0.15791413187980652, 0.0..."
3,data/encodings/homeland_2011_796960/S03E07_562...,"[-0.0779518112540245, 0.15547886490821838, 0.1..."
4,data/encodings/homeland_2011_796960/S07E04_111...,"[-0.12568266689777374, 0.1840898096561432, 0.0..."


In [41]:
tqdm().pandas()
data = df.progress_apply(parse_filename, axis=1)    

0it [00:00, ?it/s]

  0%|          | 0/712022 [00:00<?, ?it/s]

In [42]:
new_df = pd.DataFrame(data)
new_df.head()

,filepath,encoding,season,episode,frame_num,face_num
0,data/encodings/homeland_2011_796960/S05E04_390...,"[-0.11495646834373474, 0.09172723442316055, 0....",5,4,39048,0
1,data/encodings/homeland_2011_796960/S06E05_389...,"[-0.026263369247317314, 0.03598316013813019, 0...",6,5,38976,1
2,data/encodings/homeland_2011_796960/S01E01_429...,"[-0.2073737531900406, 0.15791413187980652, 0.0...",1,1,4296,0
3,data/encodings/homeland_2011_796960/S03E07_562...,"[-0.0779518112540245, 0.15547886490821838, 0.1...",3,7,56208,0
4,data/encodings/homeland_2011_796960/S07E04_111...,"[-0.12568266689777374, 0.1840898096561432, 0.0...",7,4,11112,0


In [38]:
def format_series_name(data):
    title = data['title'].lower().replace(' ', '-')
    year = data['year']
    imdb_id = data['series_id']
    name = f'{title}_{year}_{imdb_id}'
    return name

In [40]:
faces_dir = Path('./data/faces')
dst_dir = Path('./data/faces_new')
sub_dirs = [x for x in faces_dir.iterdir()]
faces = []
for sub_dir in sub_dirs[:5]:
    files = [x for x in sub_dir.iterdir()]
    temp = pd.read_csv(str(files[0]), index_col=0)
    try:
        d = dst_dir.joinpath(format_series_name(temp.iloc[0]))
    except IndexError:
        continue
    if not d.exists:
        Path.mkdir(d)
    for file in files[:5]:
        face = pd.read_csv(str(file), index_col=0)
        face = face.merge(new_df.drop('filepath', axis=1),
                          how='left',
                          on=['series_id', 'season', 'episode', 'frame_num', 'face_num'])
        if 'confidence' not in face.columns:
            face = face.assign('confidence'=np.nan)
        fp = d.joinpath(file.name)
        face.to_csv(str(fp))

KeyError: 'title'

In [43]:
def f(row):
    title, year, series_id = Path(row['filepath']).parent.parts[-1].split('_')
    row['title'] = title
    row['year'] = int(year)
    row['series_id'] = int(series_id)
    return row

In [46]:
tqdm.pandas()
rows = new_df.progress_apply(f, axis=1)
test = pd.DataFrame(rows)
test.head()

  0%|          | 0/712022 [00:00<?, ?it/s]

,filepath,encoding,season,episode,frame_num,face_num,title,year,series_id
0,data/encodings/homeland_2011_796960/S05E04_390...,"[-0.11495646834373474, 0.09172723442316055, 0....",5,4,39048,0,homeland,2011,796960
1,data/encodings/homeland_2011_796960/S06E05_389...,"[-0.026263369247317314, 0.03598316013813019, 0...",6,5,38976,1,homeland,2011,796960
2,data/encodings/homeland_2011_796960/S01E01_429...,"[-0.2073737531900406, 0.15791413187980652, 0.0...",1,1,4296,0,homeland,2011,796960
3,data/encodings/homeland_2011_796960/S03E07_562...,"[-0.0779518112540245, 0.15547886490821838, 0.1...",3,7,56208,0,homeland,2011,796960
4,data/encodings/homeland_2011_796960/S07E04_111...,"[-0.12568266689777374, 0.1840898096561432, 0.0...",7,4,11112,0,homeland,2011,796960


In [ ]:
faces_dir = Path('./data/faces')
dst_dir = Path('./data/faces_new')
sub_dirs = [x for x in faces_dir.iterdir()]
faces = []
df_a = df.assign(name=lambda x: Path(x['filepath']).stem)
for sub_dir in sub_dirs[:5]:
    files = [x for x in sub_dir.iterdir()]
    for file in files:
        face_df = pd.DataFrame(str(file), index_col=0)
        face_df = face_df.assign(name=lambda x: Path())
        encoding_df = face_df.assign(encoding_path=)
        

In [ ]:
df.iloc[0]['encoding']

In [ ]:
faces_dir = Path('./data/faces')
dst_dir = Path('./data/faces_new')
sub_dirs = [x for x in faces_dir.iterdir()]
faces = []
for sub_dir in sub_dirs[:5]:
    files = [x for x in sub_dir.iterdir()]
    temp = pd.read_csv(str(files[0]), index_col=0)
    try:
        d = dst_dir.joinpath(format_series_name(temp.iloc[0]))
    except IndexError:
        continue
    if not d.exists:
        Path.mkdir(d)
    for file in files[:5]:
        face = pd.read_csv(str(file), index_col=0)
        face = face.merge(new_df.drop('filepath', axis=1),
                          how='left',
                          on=['series_id', 'season', 'episode', 'frame_num', 'face_num'])
        if 'confidence' not in face.columns:
            face = face.assign('confidence'=np.nan)
        fp = d.joinpath(file.name)
        face.to_csv(str(fp))